# Importing the World: 3D Format Visualization and Conversion

This notebook loads, compares, and converts 3D models in .OBJ, .STL, and .GLTF formats using trimesh.
The goal is to understand the structural differences between formats and how to perform conversions.

## 1. Install and Import Libraries

In [103]:
# Install required libraries (uncomment if not installed)
# !pip install trimesh numpy open3d vedo

import trimesh
import numpy as np
import os
from pathlib import Path
import vedo

vedo.settings.default_backend = "vtk"

## 2. Model Loading Function

This function loads models in different formats and correctly handles GLTF scenes.

In [104]:
def load_model(file_path):
    """
    Load a 3D model from a file.
    
    Parameters:
    file_path : str
        Path to the model file (.obj, .stl, .gltf, .glb)
    
    Returns:
    trimesh.Trimesh : The loaded model
    """
    try:
        model = trimesh.load(file_path, force='mesh')
        
        # If the model is a scene (like GLTF), combine geometries
        if isinstance(model, trimesh.Scene):
            model = trimesh.util.concatenate(
                tuple(trimesh.Trimesh(vertices=g.vertices, faces=g.faces)
                      for g in model.geometry.values())
            )
        
        print(f"Model loaded: {Path(file_path).name}")
        return model
    
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

## 3. Detailed Property Analysis

This function analyzes and displays detailed properties of each model.

In [105]:
def analyze_model(model, format_name):
    """
    Analyze the structural properties of a 3D model.
    
    Parameters:
    model : trimesh.Trimesh
        The model to analyze
    format_name : str
        Format name for display
    """
    if model is None:
        print("No model loaded")
        return None
    
    print(f"\n{'='*60}")
    print(f"ANALYSIS: {format_name}")
    print(f"{'='*60}")
    
    # Basic information
    print(f"\nGeometric Structure:")
    print(f"   • Vertices: {len(model.vertices):,}")
    print(f"   • Faces: {len(model.faces):,}")
    print(f"   • Edges: {len(model.edges):,}")
    
    # Duplicate analysis
    print(f"\nDuplicate Analysis:")
    unique_vertices = len(np.unique(model.vertices, axis=0))
    duplicate_vertices = len(model.vertices) - unique_vertices
    print(f"   • Unique vertices: {unique_vertices:,}")
    print(f"   • Duplicate vertices: {duplicate_vertices:,}")
    
    # Normals
    print(f"\nNormals:")
    if hasattr(model, 'vertex_normals'):
        print(f"   • Vertex normals: {len(model.vertex_normals):,}")
    if hasattr(model, 'face_normals'):
        print(f"   • Face normals: {len(model.face_normals):,}")
    
    # Physical properties
    print(f"\nPhysical Properties:")
    print(f"   • Volume: {model.volume:.4f} units³")
    print(f"   • Surface area: {model.area:.4f} units²")
    print(f"   • Center of mass: ({model.center_mass[0]:.2f}, {model.center_mass[1]:.2f}, {model.center_mass[2]:.2f})")
    
    # Bounding box
    bounds = model.bounds
    print(f"\nBounding Box:")
    print(f"   • Min: ({bounds[0][0]:.2f}, {bounds[0][1]:.2f}, {bounds[0][2]:.2f})")
    print(f"   • Max: ({bounds[1][0]:.2f}, {bounds[1][1]:.2f}, {bounds[1][2]:.2f})")
    
    # Validation
    print(f"\n✓ Validation:")
    print(f"   • Is watertight: {model.is_watertight}")
    print(f"   • Is convex: {model.is_convex}")
    
    return {
        'vertices': len(model.vertices),
        'faces': len(model.faces),
        'edges': len(model.edges),
        'unique_vertices': unique_vertices,
        'duplicates': duplicate_vertices,
        'volume': model.volume,
        'area': model.area,
        'is_watertight': model.is_watertight
    }

## 4. Format Comparison

Function to compare the properties of models in different formats.

In [106]:
def compare_formats(models_dict):
    """
    Compare the properties of models in different formats.
    
    Parameters:
    models_dict : dict
        Dictionary with format as key and model as value
    """
    print(f"\n{'='*80}")
    print(f"FORMAT COMPARISON")
    print(f"{'='*80}")
    
    stats = {}
    for format_name, model in models_dict.items():
        if model is not None:
            stats[format_name] = analyze_model(model, format_name.upper())
    
    # Comparative table
    if len(stats) > 1:
        print(f"\n{'='*80}")
        print(f"COMPARATIVE SUMMARY")
        print(f"{'='*80}")
        print(f"\n{'Property':<25} | {'GLTF':<15} | {'OBJ':<15} | {'STL':<15}")
        print(f"{'-'*80}")
        
        properties = ['vertices', 'faces', 'edges', 'unique_vertices', 'duplicates']
        for prop in properties:
            row = f"{prop.replace('_', ' ').title():<25}"
            for fmt in ['gltf', 'obj', 'stl']:
                if fmt in stats and stats[fmt]:
                    value = stats[fmt].get(prop, 'N/A')
                    row += f" | {str(value):>13}"
                else:
                    row += f" | {'N/A':>13}"
            print(row)
    
    return stats

## 5. Format Conversion

Functions to convert models between different formats.

In [ ]:
def convert_format(model, output_path, target_format):
    """
    Convert a model to a different format.
    
    Parameters:
    model : trimesh.Trimesh
        Model to convert
    output_path : str
        Output path (without extension)
    target_format : str
        Target format ('obj', 'stl', 'gltf', 'glb')
    """
    if model is None:
        print("No model to convert")
        return None
    
    try:
        # Ensure output has the correct extension
        output_file = f"{output_path}.{target_format}"
        
        # Export according to format
        if target_format.lower() in ['gltf', 'glb']:
            # GLTF/GLB requires exporting as a scene
            scene = trimesh.Scene(model)
            scene.export(output_file)
        else:
            # OBJ and STL are exported directly
            model.export(output_file)
        
        print(f"✓ Converted to {target_format.upper()}: {output_file}")
        return output_file
    
    except Exception as e:
        print(f"✗ Conversion error: {e}")
        return None


def batch_convert(model, base_name, formats=['obj', 'stl', 'gltf']):
    """
    Convert a model to multiple formats.
    
    Parameters:
    model : trimesh.Trimesh
        Model to convert
    base_name : str
        Base name for files (without extension)
    formats : list
        List of target formats
    """
    print(f"\n{'='*60}")
    print(f"BATCH CONVERSION TO MULTIPLE FORMATS")
    print(f"{'='*60}\n")
    
    converted_files = {}
    for fmt in formats:
        output_file = convert_format(model, base_name, fmt)
        if output_file:
            converted_files[fmt] = output_file
    
    return converted_files

## 6. Comparative Visualization

Visualize multiple models side-by-side for comparison.

In [108]:
def visualize_comparison(models_dict):
    """
    Visualize multiple models in a grid for comparison.
    
    Parameters:
    models_dict : dict
        Dictionary with format as key and model as value
    """
    # Filter valid models
    valid_models = {k: v for k, v in models_dict.items() if v is not None}
    
    if len(valid_models) == 0:
        print("No valid models to visualize")
        return
    
    n_models = len(valid_models)
    
    # Configure layout based on number of models
    if n_models == 1:
        shape = (1, 1)
    elif n_models == 2:
        shape = (1, 2)
    else:
        shape = (1, 3)
    
    plt = vedo.Plotter(shape=shape, title="3D Format Comparison", size=(1600, 600))
    
    # Configure dark background
    for renderer in plt.renderers:
        renderer.SetBackground(0.10, 0.10, 0.10)
    
    # Display each model
    for idx, (format_name, model) in enumerate(valid_models.items()):
        mesh = vedo.Mesh([model.vertices, model.faces])
        mesh.color('lightblue').lighting('glossy')
        
        plt.at(idx).show(
            mesh, 
            f"{format_name.upper()} Format\n{len(model.vertices)} vertices, {len(model.faces)} faces",
            axes=1,
            viewup='z'
        )
    
    plt.show(interactive=True)
    return plt

## 7. Load Models in All Three Formats

Load the same model in different formats for comparison.
**Note:** You should have sample files in all three formats (.obj, .stl, .gltf/.glb)

In [109]:
# Define model paths
# Adjust these paths according to your files
model_paths = {
    'gltf': 'husky.gltf',      # or .gltf
    'obj': 'husky.obj',       # Change to your .obj file
    'stl': 'husky.stl'        # Change to your .stl file
}

# Load models
models = {}
print("Loading models...\n")
for format_name, path in model_paths.items():
    if os.path.exists(path):
        models[format_name] = load_model(path)
    else:
        print(f"⚠ File not found: {path}")
        models[format_name] = None

Loading models...

Model loaded: husky.gltf
Model loaded: husky.obj
Model loaded: husky.stl


## 8. Analysis and Comparison

Analyze the properties of each model and compare the formats.

In [110]:
# Compare formats
stats = compare_formats(models)


FORMAT COMPARISON

ANALYSIS: GLTF

Geometric Structure:
   • Vertices: 8,424
   • Faces: 468
   • Edges: 1,404

Duplicate Analysis:
   • Unique vertices: 310
   • Duplicate vertices: 8,114

Normals:
   • Vertex normals: 8,424
   • Face normals: 468

Physical Properties:
   • Volume: 1285.2304 units³
   • Surface area: 1995.5588 units²
   • Center of mass: (0.00, 8.86, -1.75)

Bounding Box:
   • Min: (-3.81, 0.00, -15.07)
   • Max: (3.81, 15.31, 17.91)

✓ Validation:
   • Is watertight: False
   • Is convex: False

ANALYSIS: OBJ

Geometric Structure:
   • Vertices: 936
   • Faces: 468
   • Edges: 1,404

Duplicate Analysis:
   • Unique vertices: 310
   • Duplicate vertices: 626

Normals:
   • Vertex normals: 936
   • Face normals: 468

Physical Properties:
   • Volume: 1285.2304 units³
   • Surface area: 1995.5587 units²
   • Center of mass: (0.00, 8.86, -1.75)

Bounding Box:
   • Min: (-3.81, 0.00, -15.07)
   • Max: (3.81, 15.31, 17.91)

✓ Validation:
   • Is watertight: False
   • Is 

## 9. Convert Between Formats

Example: Convert the GLTF model to other formats.

In [111]:
# If you have a GLTF model, convert it to other formats
if models.get('gltf') is not None:
    converted = batch_convert(models['gltf'], 'converted_model', ['obj', 'stl'])
    
    # Load converted models to verify
    print("\n" + "="*60)
    print("CONVERTED MODELS VERIFICATION")
    print("="*60)
    
    for fmt, file_path in converted.items():
        if os.path.exists(file_path):
            converted_model = load_model(file_path)
            if converted_model:
                analyze_model(converted_model, f"Converted to {fmt.upper()}")


BATCH CONVERSION TO MULTIPLE FORMATS

✓ Converted to OBJ: converted_model.obj
✓ Converted to STL: converted_model.stl

CONVERTED MODELS VERIFICATION
Model loaded: converted_model.obj

ANALYSIS: Converted to OBJ

Geometric Structure:
   • Vertices: 936
   • Faces: 468
   • Edges: 1,404

Duplicate Analysis:
   • Unique vertices: 310
   • Duplicate vertices: 626

Normals:
   • Vertex normals: 936
   • Face normals: 468

Physical Properties:
   • Volume: 1285.2304 units³
   • Surface area: 1995.5588 units²
   • Center of mass: (0.00, 8.86, -1.75)

Bounding Box:
   • Min: (-3.81, 0.00, -15.07)
   • Max: (3.81, 15.31, 17.91)

✓ Validation:
   • Is watertight: False
   • Is convex: False
Model loaded: converted_model.stl

ANALYSIS: Converted to STL

Geometric Structure:
   • Vertices: 310
   • Faces: 468
   • Edges: 1,404

Duplicate Analysis:
   • Unique vertices: 310
   • Duplicate vertices: 0

Normals:
   • Vertex normals: 310
   • Face normals: 468

Physical Properties:
   • Volume: 1285.

## 10. Comparative Visualization

Visualize all loaded models side-by-side.

In [112]:
# Visualize loaded models
if any(models.values()):
    visualize_comparison(models)
else:
    print("No valid models to visualize")